In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from pathlib import Path

import precision_functions as prefun

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 300)

### Constants

In [ ]:
EVENT = "rastoc:stillness-position-lost"
TRIAL_TAG = 'validation-stimulus' #Opcion: "fixation-stimulus"  # Es el momento donde aparece la cruz de fijacion
FIRST_SAMPLE = 500
MAX_VAR = 75
MAX_PLOTS = None
VERBOSE = False
ALL_FILES_RAW = list(Path("./raw_data").glob("*.csv"))
LOAD_PATH = './raw_data/juan_repeticion_webcam9_computer3_chrome_ubuntu.csv' #str(ALL_FILES_RAW[0])
# LOAD_INFO = LOAD_PATH.split('/')[1].split('_')


# print("All files:",ALL_FILES_RAW, end='\n\n')
# print(f"""File loaded:
# {LOAD_INFO[0]}
# {LOAD_INFO[1]}
# {LOAD_INFO[2]}
# {LOAD_INFO[3]}
# {LOAD_INFO[4].split('.')[0]}""")


### Data load

In [ ]:
print(LOAD_PATH)
df = pd.read_csv(LOAD_PATH)
df.head(5)

### Check rastoc events

In [ ]:
events  = prefun.get_rastoc_events(df, EVENT)
print("len(events):", len(events))

### Precision experiment analysis

In [ ]:
df_res = prefun.calculate_and_plot_errors(df, 
                                   TRIAL_TAG,
                                   first_sample=FIRST_SAMPLE,
                                   max_var=MAX_VAR,
                                   max_plots=MAX_PLOTS, 
                                   verbose=VERBOSE)

print(f"Error: {df_res['total_errors_pxs_mean'].mean():.2f} +- {df_res['total_errors_pxs_mean'].std():.2f}")

### Analysis

In [ ]:
print(f"Error: {df_res['total_errors_pxs_mean'].mean():.2f} +- {df_res['total_errors_pxs_mean'].std():.2f}")
print(f"Cantidad de NaNs: {df_res['total_errors_pxs_mean'].isna().sum()}")


In [ ]:
plt.plot((df_res['trials']-1)%8,df_res['total_errors_pxs_mean'],'.')

In [ ]:
# plt.plot((df_res['trials']-1)%8,df_res['total_errors_pxs_mean'],'.')

sns.regplot(df_res, x=(df_res['trials']-1)%8, y='total_errors_pxs_mean')

# a = (df_res['trials']-1)%8
# sns.boxplot(data=df_res, x=a, y='total_errors_pxs_mean')

In [ ]:
a = (df_res['trials']-1)%8
# sns.regplot(df_res, x=a, y='total_errors_pxs_mean')
ax = sns.scatterplot(df_res,  x=a, y='total_errors_pxs_mean', hue='presented_point')
ax.legend(bbox_to_anchor=(1.3, 1.3))

In [ ]:
center_x = df[df["trial-tag"] == TRIAL_TAG]["center_x"].iloc[0]
center_y = df[df["trial-tag"] == TRIAL_TAG]["center_y"].iloc[0]

df_res['presented_distance'] = df_res['presented_point'].apply(lambda x: np.sqrt((x[0] - center_x)**2 + (x[1] - center_y)**2))

df_res.groupby('presented_distance').mean()



In [ ]:
plt.scatter(df_res['webgazer_x'].iloc[10], df_res['webgazer_y'].iloc[10])
plt.xlim(0, 1920)
plt.ylim(0, 1080)

In [ ]:
df_res.plot.scatter(x='presented_point_x',y='presented_point_y')

In [ ]:
df_res[['presented_point_x','presented_point_y']] = pd.DataFrame(df_res['presented_point'].tolist())

In [ ]:
points_position_dict = dict(zip(df_res['presented_point'].unique(), range(1,10)))

df_res['presented_point_index'] = df_res["presented_point"].map(points_position_dict)

In [ ]:
df_res[['presented_point_index', 'presented_point']].head(20)

In [ ]:
# U = df_res['presented_point'].unique()
# U

# [idx for x,y in df_res['presented_point'] ]

df.columns

In [ ]:
# plt.plot(df_res['trials'],df_res['total_errors_pxs_mean'],'.')
sns.regplot(df_res, x='trials', y='total_errors_pxs_mean')


In [ ]:
(8-1)%8

In [ ]:
ALL_FILES_ANALYZED = list(Path("./analyzed").glob("*.csv"))
ALL_FILES_ANALYZED

prefun.evaluate_experiment_instances(ALL_FILES_ANALYZED)

In [ ]:
df_res['first_sample'].iloc[0]

### Save

In [ ]:
SAVE_PATH = f"analyzed/analyzed_{LOAD_PATH.split('/')[-1]}"
print(f'Saved: {SAVE_PATH}')
df_res.to_csv(SAVE_PATH, index=False)

### Cosas

In [ ]:

df = pd.read_csv('/home/gus/Downloads/antisacadas_91.csv')
df.head(5)

In [ ]:
df.columns

In [ ]:
# df[~df['webgazer_data'].isna()]['webgazer_data']

# Antisacadas
df_antisacadas = df.query("not webgazer_data.isnull() and not typeOfSaccade == 'antisaccade'")

webgazer_data_antisacadas = df_antisacadas['webgazer_data'].map(eval)
for trial in range(len(df_antisacadas)):
    xs = [i['x'] for i in webgazer_data_antisacadas.iloc[trial]]
    ys = [i['y'] for i in webgazer_data_antisacadas.iloc[trial]]
    ts = [i['t'] for i in webgazer_data_antisacadas.iloc[trial]]
    plt.plot(ts, xs)

plt.title(f'antisacadas\nN={len(webgazer_data_antisacadas)}')
plt.ylabel('x coordinate (pxs)')
plt.xlabel('time (ms)')
plt.show()

# Prosacadas
df_prosacadas = df.query("not webgazer_data.isnull() and not typeOfSaccade == 'prosaccade'")

webgazer_data_prosacadas = df_prosacadas['webgazer_data'].map(eval)
for trial in range(len(df_antisacadas)):
    xs = [i['x'] for i in webgazer_data_prosacadas.iloc[trial]]
    ys = [i['y'] for i in webgazer_data_prosacadas.iloc[trial]]
    ts = [i['t'] for i in webgazer_data_prosacadas.iloc[trial]]
    plt.plot(ts, xs)

plt.title(f'Prosacadas\nN={len(webgazer_data_prosacadas)}')
plt.ylabel('x coordinate (pxs)')
plt.xlabel('time (ms)')
plt.show()

In [ ]:
trial = 100
xs = [i['x'] for i in df[~df['webgazer_data'].isna()]['webgazer_data'].map(eval).iloc[trial]]
ys = [i['y'] for i in df[~df['webgazer_data'].isna()]['webgazer_data'].map(eval).iloc[trial]]
ts = [i['t'] for i in df[~df['webgazer_data'].isna()]['webgazer_data'].map(eval).iloc[trial]]

plt.plot(ts, xs)
# plt.xlim(0, int(df[~df['webgazer_data'].isna()]['viewportWidth'].iloc[-1]))
# plt.ylim(0, df[~df['webgazer_data'].isna()]['viewportHeight'].iloc[-1])
# plt.colorbar()
plt.show()